###### ===================================================================================================================

# SQL Connection File

###### ===================================================================================================================

### Run Command

In [1]:
%run Libraries.ipynb

### Class: SQL

In [2]:
class SQL:
    def __init__(self, driver, server, database):
        """
        Initializes the SQL class with connection details.

        Args:
            driver (str): The ODBC driver name.
            server (str): The SQL server name or IP address.
            database (str): The name of the SQL database.
        """
        self.driver = driver
        self.server = server
        self.database = database
        
    def Fetch_DataFrame(self, query):
        """
        Fetches data from the SQL database and returns it as a DataFrame.

        Args:
            query (str): The SQL query to execute.

        Returns:
            pd.DataFrame or None: The fetched data as a DataFrame, or None if an error occurs.
        """
        try:
            # Connection String
            cxn = pyodbc.connect('DRIVER=' + self.driver + ';'
                                'SERVER=' + self.server + ';'
                                'DATABASE=' + self.database + ';'
                                'Trusted_Connection=yes')

            # Cursor
            cursor = cxn.cursor()
            cursor.execute(query)
            
            # DataFrame preparation
            rows = cursor.fetchall()
            extracted_rows = [tuple(cell[0] if isinstance(cell, list) else cell for cell in row) for row in rows]
            column_names = [column[0] for column in cursor.description]
            
            # Closing connection instance
            cxn.close()
            
            return pd.DataFrame(extracted_rows, columns=column_names)
        
        except Exception as e:
            # Handle exceptions here, or at least log them for debugging
            print(f"An error occurred while fetching the dataframe: {e}")
            
            return None

    def Append_SQL_Table(self, table_name, current_date, dataframe):
        """
        Appends data from a DataFrame to an SQL table.

        Args:
            table_name (str): The name of the SQL table.
            current_date (str): The date for which data should be appended.
            dataframe (pd.DataFrame): The DataFrame containing data to be appended.
        """
        try:
            # Connection String
            cxn = pyodbc.connect('DRIVER=' + self.driver + ';'
                                 'SERVER=' + self.server + ';'
                                 'DATABASE=' + self.database + ';'
                                 'Trusted_Connection=yes')

            # Cursor
            cursor = cxn.cursor()
            if current_date is not None:
                # Deleting duplicate data for the same date
                drop_existing_values_if_exist_query = f"DELETE FROM [{table_name}] WHERE Date = '{current_date}'"
                cursor.execute(drop_existing_values_if_exist_query)
                cursor.commit()
            else:
                pass

            # Fetching table schema
            columns_query = f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}' ORDER BY COLUMN_NAME"
            cursor.execute(columns_query)
            columns_info = cursor.fetchall()
            
            table_columns = [column[0] for column in columns_info]
            # Creating a list of the common columns
            common_columns = [col for col in dataframe.columns if col in table_columns]
            column_list = ', '.join(common_columns)
            placeholders = ', '.join('?' for _ in common_columns)
            
            # Query to enter values into the table
            query = f"INSERT INTO {table_name} ({column_list}) VALUES ({placeholders})"

            dataframe_subset = dataframe[common_columns]

            # DataFrame preparation
            prepared_data = []
            for row in dataframe_subset.itertuples(index=False):
                prepared_row = []
                for i, value in enumerate(row):
                    column_name = common_columns[i]
                    data_type = next((column[1] for column in columns_info if column[0] == column_name), None)
                    if isinstance(value, pd.Timestamp):
                        value = value.to_pydatetime()
                    elif pd.isna(value):
                        value = None
                    elif data_type == 'decimal':
                        value = decimal.Decimal(str(value))
                    elif data_type == 'float':
                        value = float(value)
                    elif data_type == 'int':
                        value = int(value)
                    prepared_row.append(value)
                prepared_data.append(prepared_row)

            if prepared_data:
                cursor.executemany(query, prepared_data)
                cursor.commit()
            
            else:
                pass
            
        except Exception as e:
            # Handle exceptions here, or at least log them for debugging
            print(f"An error occurred while appending the MSSQL table: {e}")
        
        finally:
            # Closing connection instance
            cursor.close()
            cxn.close()
            
    def Read_DataFrame(self, table_name, current_date=None):
    """
    Reads data from an SQL table and returns it as a DataFrame.

    Args:
        table_name (str): The name of the SQL table to read data from.
        current_date (str, optional): The current date for filtering data by date. Default is None.

    Returns:
        pd.DataFrame or None: The fetched data as a DataFrame, or None if an error occurs.
    """
    try:    
        # Create a list of MSSQL tables with yearly input data
        list_table_names = [
            'StockPriceData',
            'IndexPerformanceData',
            'CommodityPriceData',
            'BondPriceData',
            'EquityInvestments',
            'MutualFundInvestments',
            'DebtInvestments',
            'CommodityInvestments',
            'TotalInvestments',
            'UserMetrics',
            'EquityMetrics',
            'MutualFundMetrics',
            'DebtMetrics',
            'CommodityMetrics'
        ]
        
        if table_name in list_table_names:
            query = f"""
                SELECT 
                    * 
                FROM
                    {table_name}
                WHERE
                    Date BETWEEN DATEADD(year, -5, '{current_date}') AND '{current_date}'
                """
        
        else:
            query = f"SELECT * FROM {table_name}"
        
        # Fetch data from SQL Server and read the input_data dataframe
        dataframe = self.Fetch_DataFrame(query)
        
        return dataframe
    
    except Exception as e:
        # Handle exceptions here, or at least log them for debugging
        print(f"An error occurred while reading the dataframe: {str(e)}")
        
        return None

### Connection Details

In [3]:
# Connection details
Driver = '{SQL Server}'
Server = 'DESKTOP-UM221JG\SQLEXPRESS'
Database = 'WealthManagement'

# Create an instance of the SQL class
SQL_Connector = SQL(Driver, Server, Database)        

# Defining the current date
Current_Date = date.today()